#### merge urls to recipe dataframe

In [660]:
# import relevant libraries
import pandas as pd
import numpy as np
from collections import Counter
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import ast
import re
import math

In [661]:
# recipes dataframes
# there is a mismatch in df3_urls and df3_recipes 2430 vs 1022
df1 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df1.csv')
df2 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df2.csv')
df3 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df3.csv')
df4 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df4.csv')
df5 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df5.csv')
df6 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df6.csv')
df7 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df7.csv')
df8 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df8.csv')

In [662]:
# concat the dataframes
df_recipes = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
df_recipes = df_recipes.drop(['Unnamed: 0','id'], axis=1) #drop columns not required

In [663]:
df_recipes=df_recipes.reset_index(drop='index')

In [664]:
# url dataframes
df1_url = pd.read_csv('../ScrapeData/csv_urls/urls_df1.csv')
df2_url = pd.read_csv('../ScrapeData/csv_urls/urls_df2.csv')
df3_url = pd.read_csv('../ScrapeData/csv_urls/urls_df3.csv')
df3_url = df3_url.iloc[:1022,:]
df4_url = pd.read_csv('../ScrapeData/csv_urls/urls_df4.csv')
df5_url = pd.read_csv('../ScrapeData/csv_urls/urls_df5.csv')
df6_url = pd.read_csv('../ScrapeData/csv_urls/urls_df6.csv')

In [665]:
# concat the dataframes url and recipes
df_url = pd.concat([df1_url,df2_url, df3_url, df4_url,df5_url,df6_url])
df_url = df_url.drop(['Unnamed: 0'], axis=1) #drop columns not required

In [666]:
df_url = df_url.reset_index(drop='index')

In [667]:
# merge df_url and df_recipes
df = pd.concat([df_url,df_recipes],axis=1)

#### preprocess the dataframe

In [668]:
df['makeitagainscore'] = pd.to_numeric(df['makeitagainscore'].str.replace('%', ''))

In [669]:
# drop rows if there are null value records
df.dropna(axis=0, how='any', inplace=True,subset=['bestrating','ingredients',\
                            'makeitagainscore','preparation','rating','reviews','tags','title'])

In [670]:
df["ingredients_refined"] = ""
for index, row in df.iterrows():
    ingredients_items = ast.literal_eval(row["ingredients"])
    df.set_value(index,'ingredients_refined',ingredients_items)

#### extract ingrediants

In [671]:
# list of prepositions
prepositions = ["A", "An", "a", "about", "above", "above", "across", "additional","after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "plus", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thick", "thickly", "thin", "thinly", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [672]:
# Common measurement words and hard-coded words.
measurement_words = ['pound',
 'cubes',
  'cups',
  'cut',
 'kilogram',
 'ounce','-ounce',
 'gram',
 'cup',
 'spoon',
 'quart',
 'teaspoon','teaspoons',
 'smidgen',
 'drop',
 'gallon',
 'dash',
 'handful',
 'scoop',
 'bowl',
 'inch',
 '-inch',
 'pinch',
 'liter',
 'milliliter',
 'tablespoon', 'tablespoons',
 'fluid',
 'bottle',
 'tad',
 'pint', 'Tbsp', 'tsp','lb']

In [673]:
# create a list of stopwords, prepositions, and measurement words
filtered = nltk.corpus.stopwords.words('english')
for word in prepositions:
    filtered.append(word)
for word in measurement_words:
    filtered.append(word)

In [674]:
df["ingredient_list"] = ""
for index, row in df.iterrows():
    ingredients_items = ast.literal_eval(row["ingredients"])

    ingredients = []
    pattern = "([^\d\\.\\,\\\;\\*\\]\\[\\%\\½\\<\\>\\:\\°\\?\\�\\}\\{\)\(\"\/\s]+)"
    for ingredient in ingredients_items:
        p = re.compile(pattern)
        result = p.findall(ingredient) 
        ingredients.append(result)
        
        
    single_ingr = sum(ingredients, [])
    recipe_ingr = list(set(single_ingr))


    filtered_ingr = []
    for item in recipe_ingr:
        if item not in stopWords:
            filtered_ingr.append(item)
    df.set_value(index,"ingredient_list",filtered_ingr)

In [675]:
#export the refined dataframe to a csv
df.to_csv('df_similaritly.csv')

#### similarity script

In [626]:
sample_str = [
    "rice", "carrots", "shallots"
]

In [627]:
counterA = Counter(sample_str)
counterB = Counter(df['ingredient_list'][0])

In [658]:
def counter_cosine_similarity(c1, c2):
    '''a function to calculate the cosine similarity between two lists'''
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [629]:
df['similarity_score']=""

for index, row in df.iterrows():
    try:
        counterB = Counter(row['ingredient_list'])
        score = counter_cosine_similarity(counterA,counterB)
        df.set_value(index,'similarity_score',score)
    except ZeroDivisionError:
        df.set_value(index,'similarity_score',0)

In [630]:
df.sort_values(by=['similarity_score','rating','makeitagainscore'],ascending=False).iloc[0]['urls']

'https://www.epicurious.com/recipes/food/views/carrot-ginger-dressing-233572'

In [631]:
results = df.sort_values(by=['similarity_score','rating','makeitagainscore'],ascending=[False,False,False])[:3]

In [632]:
results = results[['urls','ingredients_refined','title','rating','similarity_score']]

In [645]:
results = results.reset_index(drop='index')

In [646]:
results.to_json(orient='index')

'{"0":{"urls":"https:\\/\\/www.epicurious.com\\/recipes\\/food\\/views\\/carrot-ginger-dressing-233572","ingredients_refined":["1\\/2 lb carrots (3 medium), coarsely chopped","1\\/4 cup chopped peeled fresh ginger","1\\/4 cup chopped shallots","1\\/4 cup seasoned rice vinegar","1 tablespoon soy sauce","1 tablespoon Asian sesame oil","1\\/8 teaspoon salt","1\\/2 cup vegetable oil","1\\/4 cup water"],"title":"Carrot Ginger Dressing ","rating":3.5,"similarity_score":0.3973597071},"1":{"urls":"https:\\/\\/www.epicurious.com\\/recipes\\/food\\/views\\/truffled-lobster-risotto-4622","ingredients_refined":["2 8-ounce uncooked lobster tails","3 1\\/2 cups canned low-salt chicken broth","3 tablespoons white or black truffle oil","3\\/4 cup chopped peeled carrots","1\\/4 cup chopped shallots","1 cup arborio rice","1\\/4 cup brandy","1\\/3 cup whipping cream","1\\/3 cup chopped fresh chives"],"title":"Truffled Lobster Risotto ","rating":3.5,"similarity_score":0.3692744729},"2":{"urls":"https:\\/\